In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../Data/selected_data/processed_data.csv')

# merging medium bowlers to fast
data.loc[data['bowler_type'] == 'Left arm Medium', 'bowler_type'] = 'Left arm Fast'
data.loc[data['bowler_type'] == 'Right arm Medium', 'bowler_type'] = 'Right arm Fast'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   batter               217242 non-null  object 
 1   bowler               217242 non-null  object 
 2   non_striker          217242 non-null  object 
 3   runs_by_bat          217242 non-null  int64  
 4   extra_runs           217242 non-null  int64  
 5   total_runs_delivery  217242 non-null  int64  
 6   current_team_total   217242 non-null  int64  
 7   runs_remain          217242 non-null  float64
 8   batter_runs          217242 non-null  int64  
 9   balls_faced          217242 non-null  int64  
 10  wickets_fallen       217242 non-null  int64  
 11  extra_type           217242 non-null  object 
 12  delivery             217242 non-null  float64
 13  over                 217242 non-null  int64  
 14  wicket_type          217242 non-null  object 
 15  player_out       

In [101]:
def getPlayerScores(player_name: str, innings: list[int] = [1, 2] ) -> pd.DataFrame:
    # Get the data for BKG Mendis if batter is BKG Mendis or non-striker is BKG Mendis
	player_data = data.loc[
		((data['batter'] == player_name) | (data['non_striker'] == player_name)) & (data['innings'].isin(innings))
	]

	player_data.head()

	# 3 matches missing from the data
	# group data by match_id
	gp = player_data.groupby('match_id')
	cols = [ 'batter', 'non_striker', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery', 'wickets_fallen', 'bowling_team']
	player_scores= gp.last().loc[:, cols]

	# get the first ball he faced or at non-striker
	first_ball = gp.first().loc[:, ['over', 'delivery', 'wickets_fallen']]
	first_ball['first_ball'] = (first_ball['over'] * 6 + first_ball['delivery']).astype(int)

	player_scores['first_ball'] = first_ball['first_ball']
	player_scores['wickets_fallen'] = first_ball['wickets_fallen']

	# when BKG Mendis is the non-striker when the last ball was bowled
	# The batter_runs and balls_faced are not his, but the on_strike batter's
	# So, we need to get the last ball he faced
	# he might not even have faced a ball

	# get the last ball he faced

	matches_non_striker = player_scores[player_scores['non_striker'] == player_name].index
	
	# Sometimes the player might not even have faced a single ball
	# Eg: Afghanistan_Sri Lanka_2022-11-01 MD Shanaka not out on the non strikers end

	player_scores.loc[matches_non_striker, ['batter_runs', 'balls_faced']] = [0, 0]
	
	
	# get the last batter == player_name row from gp data
	gp = player_data[(player_data['batter'] == player_name) & (player_data['match_id'].isin(matches_non_striker))].groupby(['match_id'])
	last_batter_scores = gp.last()[['batter_runs', 'balls_faced']]	
	
	# update the rows with non_striker with correct values
	player_scores.update(last_batter_scores)
	
	



	# adding new features
	# strike rate
	player_scores['strike_rate'] = round(player_scores['batter_runs'] / player_scores['balls_faced'] * 100, 2)
	player_scores['out'] = player_scores['wicket_type'] != '0'
	player_scores['last_ball'] = (player_scores['over'] * 6 + player_scores['delivery']).astype(int)

	# drop over and delivery
	player_scores.drop(['over', 'delivery'], inplace=True, axis=1)


	# concatenating the remaining bowler types number to the dataset
	matches = data[data['match_id'].isin(player_scores.index)]
	# matches = matches[matches['batting_team'] == 'Sri Lanka']
	cols = ['match_id', 'batter', 'non_striker', 'bowler_type', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery', 'wickets_fallen']
	matches = matches[cols]
	matches['ball_number'] = (matches['over'] * 6 + matches['delivery']).astype(int)
	matches.drop(['over', 'delivery'], inplace=True, axis = 1)


	def filter_by_player_and_ball_number(group):
		player_data = group[group['batter'] == player_name]
		
		if player_data.empty:
			return player_data.drop('match_id', axis=1)
		
		first_ball_number = player_data['ball_number'].iloc[0]

		# return player_data[player_data['ball_number'] > first_ball_number].drop('match_id', axis=1) # This is for number of balls faced
		
		# fixed error should be greater or equal
		remaining = group[group['ball_number'] >= first_ball_number].drop('match_id', axis=1) # return the remianing number of deliveries for each type
		return remaining

	gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
	remaining_ball_types = gp.groupby('match_id')['bowler_type'].value_counts().unstack(fill_value = 0)
	remaining_ball_types = remaining_ball_types.reset_index()

	player_scores = player_scores.merge(remaining_ball_types, how='left', on='match_id')

	### Attaching the player_v_bowler results to this dataset
	player_vs_bowler = get_player_v_bowlers(player_name)
	


	bowler_types = ['Left arm Fast', 'Right arm Fast', 'Left arm Orthodox', 'Left arm Wrist spin', 'Right arm Legbreak', 'Right arm Offbreak']
	for bowler_type in bowler_types:
			if bowler_type in player_vs_bowler.index:
				player_scores[f'{bowler_type} Expected Runs'] = player_vs_bowler.loc[bowler_type, 'strike_rate'] / 100 *  player_scores[bowler_type]
				player_scores[f'{bowler_type} Expected Wickets'] = player_scores[bowler_type] / player_vs_bowler.loc[bowler_type, 'deliveries_per_wicket']
			else:
				player_scores[bowler_type] = 0
				player_scores[f'{bowler_type} Expected Runs'] = 0
				player_scores[f'{bowler_type} Expected Wickets'] = 0
	
	# player_scores.fillna(0, inplace=True)
	return player_scores

def get_player_v_bowlers(player_name: str, innings = [1, 2]) -> pd.DataFrame:
	player_data = data.loc[
			(data['batter'] == player_name) & (data['innings'].isin(innings))
		]

	player_data.head()

	# Convert all medium bowlers to fast
	player_data.loc[player_data['bowler_type'] == 'Left arm Medium', 'bowler_type'] = 'Left arm Fast'
	player_data.loc[player_data['bowler_type'] == 'Right arm Medium', 'bowler_type'] = 'Right arm Fast'


	player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out') 


	cols = ['batter', 'non_striker', 'runs_by_bat', 'out', 'won', 'innings', 'over', 'delivery','bowler_type']
	player_data = player_data[cols]

	gp = player_data.groupby('bowler_type')

	player_v_bowler = pd.DataFrame()
	player_v_bowler['strike_rate'] = round(gp['runs_by_bat'].mean() * 100, 3)
	player_v_bowler['strike_rate_std'] = gp['runs_by_bat'].std()
	player_v_bowler['wickets'] = gp['out'].sum()
	player_v_bowler['deliveries_per_wicket'] = round(1 / gp['out'].mean(), 3)
	player_v_bowler['deliveries'] = gp.size()
	return player_v_bowler



C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


,match_id,batter,non_striker,batter_runs,balls_faced,wicket_type,won,innings,wickets_fallen,bowling_team,...,Left arm Orthodox,Left arm Orthodox Expected Runs,Left arm Orthodox Expected Wickets,Left arm Wrist spin,Left arm Wrist spin Expected Runs,Left arm Wrist spin Expected Wickets,Right arm Legbreak Expected Runs,Right arm Legbreak Expected Wickets,Right arm Offbreak Expected Runs,Right arm Offbreak Expected Wickets
0,England_India_2018-07-03,DJ Willey,LE Plunkett,3,4,0,0.0,1,8,India,...,0,0,0,0,0,0,0,0,0.0,0.0
1,England_India_2018-07-08,LE Plunkett,CJ Jordan,9,4,caught,0.0,1,7,India,...,0,0,0,0,0,0,0,0,0.0,0.0
2,England_Pakistan_2015-11-27,LE Plunkett,DJ Willey,1,2,caught,1.0,1,6,Pakistan,...,0,0,0,0,0,0,0,0,0.0,0.0
3,England_Pakistan_2016-09-07,MM Ali,LE Plunkett,0,0,0,0.0,1,7,Pakistan,...,0,0,0,0,0,0,0,0,NaN,NaN
4,England_South Africa_2017-06-23,LA Dawson,LE Plunkett,0,0,0,0.0,2,6,South Africa,...,0,0,0,0,0,0,0,0,NaN,NaN
5,England_South Africa_2017-06-25,LE Plunkett,JC Buttler,0,2,caught,1.0,1,5,South Africa,...,0,0,0,0,0,0,0,0,0.0,0.0
6,England_Sri Lanka_2018-10-27,CJ Jordan,LE Plunkett,7,5,0,1.0,1,7,Sri Lanka,...,0,0,0,0,0,0,0,0,0.0,0.0
7,England_West Indies_2016-04-03,LE Plunkett,CJ Jordan,4,4,caught,0.0,1,8,West Indies,...,0,0,0,0,0,0,0,0,0.0,0.0
8,India_England_2017-02-01,LE Plunkett,AU Rashid,0,1,bowled,0.0,2,6,India,...,0,0,0,0,0,0,0,0,0.0,0.0
9,New Zealand_England_2018-02-13,LE Plunkett,DJ Willey,0,1,bowled,0.0,2,8,New Zealand,...,0,0,0,0,0,0,0,0,0.0,0.0


In [87]:
selected_batters = ["PWH de Silva",'KIC Asalanka','BKG Mendis',"P Nissanka",'PHKD Mendis','S Samarawickrama','AD Mathews','MD Shanaka','DM de Silva','M Theekshana','PVD Chameera','N Thushara','M Pathirana','D Madushanka']

merged_df = pd.DataFrame()

for player in data['batter'].unique():
    print("Analyzing Player", player)
    
    player_scores = getPlayerScores(player)
    player_scores['batter'] = player
    player_scores.drop('non_striker', inplace=True, axis = 1)
    merged_df = pd.concat([merged_df, player_scores])



Analyzing Player AJ Finch


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player M Klinger
Analyzing Player TM Head


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player MC Henriques
Analyzing Player AJ Turner


...

Analyzing Player LA Dawson
Analyzing Player M Mosehle
Analyzing Player AL Phehlukwayo
Analyzing Player M Morkel
Analyzing Player D Paterson


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWith

Analyzing Player DJ Malan
Analyzing Player LE Plunkett


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\3643444084.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


KeyError: 'Right arm Offbreak'

In [26]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 0 to 38
Data columns (total 31 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   match_id                              414 non-null    object 
 1   batter                                414 non-null    object 
 2   batter_runs                           414 non-null    int64  
 3   balls_faced                           414 non-null    int64  
 4   wicket_type                           414 non-null    object 
 5   won                                   414 non-null    float64
 6   innings                               414 non-null    int64  
 7   wickets_fallen                        414 non-null    int64  
 8   bowling_team                          414 non-null    object 
 9   first_ball                            414 non-null    int32  
 10  strike_rate                           414 non-null    float64
 11  out                      

In [27]:

targets = ['batter_runs', 'balls_faced', 'last_ball', 'strike_rate']
target_df = merged_df[targets]

input_df = merged_df.drop(targets, axis=1)
input_df.drop(['wicket_type', 'out', 'match_id', 'won'], inplace=True, axis = 1)

# Preprocess the data
input_df = pd.get_dummies(data=input_df,dtype=int)


In [28]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

batter_runs = target_df['balls_faced']
tree_reg = DecisionTreeRegressor()

scores = cross_val_score(tree_reg, input_df, batter_runs, scoring="neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores)



display_scores(tree_rmse_scores)
    


Scores:  [11.23875099 21.08260398 21.5296031  22.38462483 22.98090406 15.27791328
 15.97101032 14.53591012 13.56735681 14.31100549]
Mean:  17.287968295803275
Standard Deviation:  4.04256060212877


In [30]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()


SyntaxError: invalid syntax (3355457040.py, line 3)